In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
all_df = pd.read_csv('./restaurant_data_2.csv')

In [3]:
all_df

,Unnamed: 0,Unnamed: 0.1,name,address,survival for 2 or more years
0,0,0,빠레뜨한남 샤로수길점,서울특별시 관악구 남부순환로230길 40 1층,X
1,1,1,샤로수길맛집 고기굽는사람들,서울특별시 관악구 관악로14길 70 효림빌딩,X
2,2,2,동경산책,서울특별시 관악구 관악로14길 30,O
3,3,3,서울테이블,서울특별시 관악구 관악로14길 28 2층,O
4,4,4,베이컨시,서울특별시 관악구 봉천로62길 5 B1,X
...,...,...,...,...,...
245,245,245,낙성대우동,서울특별시 관악구 봉천로 576,X
246,246,246,라화쿵부마라탕,서울특별시 관악구 관악로 174 2층 2호,X
247,247,247,우동요츠야,서울특별시 관악구 관악로14길 35 지하 1층,O
248,248,248,어멍,서울특별시 관악구 낙성대로 22-5,X


In [4]:
x_rest = list(all_df[all_df['survival for 2 or more years']=='X']['name'])
review_list = []

In [5]:
for x in x_rest:
    try:
        driver = webdriver.Chrome("./chromedriver\/chromedriver.exe")
        driver.get("https://map.naver.com/v5/search")
        time.sleep(3)
        search_box = driver.find_element_by_xpath('/html/body/app/layout/div[3]/div[2]/shrinkable-layout/div/app-base/search-input-box/div/div[1]/div/input')
        search_box.send_keys(x)
        search_box.send_keys(Keys.ENTER)
        time.sleep(5)
        driver.switch_to.frame("entryIframe")
        review_list.append([x_rest.index(x),x, driver.find_element_by_xpath('//*[@id="app-root"]/div/div[2]/div[1]/div/div/div[1]').text])
        driver.quit()
    except:
        driver.quit()
        review_list.append([x_rest.index(x),x, "클릭 필요"])

In [8]:
review_list_2 = review_list
error_count=0
for li in review_list:
    if li[2] == "클릭 필요":
        try:
            now_rest = li[1]
            driver = webdriver.Chrome("./chromedriver\/chromedriver.exe")
            driver.get("https://map.naver.com/v5/search")
            time.sleep(3)
            search_box = driver.find_element_by_xpath('/html/body/app/layout/div[3]/div[2]/shrinkable-layout/div/app-base/search-input-box/div/div[1]/div/input')
            search_box.send_keys(now_rest)
            search_box.send_keys(Keys.ENTER)
            time.sleep(3)
            driver.find_element_by_xpath('//*[@id="sidebar"]/navbar/perfect-scrollbar/div/div[3]/div').click()
            driver.switch_to.frame("searchIframe")
            click_name = driver.find_elements_by_xpath('/html/body/div[3]/div/div/div[1]/ul/li/div[2]/div[1]/a/span[1]')
            text_name = []
            for i in range(len(click_name)):
                text_name.append(click_name[i].text)
            adress = driver.find_elements_by_xpath('/html/body/div[3]/div/div/div[1]/ul/li/div[2]/div[2]/span/a/span[1]')
            for i in range(len(adress)):
                adress[i] = adress[i].text
            data = [] 
            for x, y in zip(text_name, adress):
                data.append([x, y])
            def rest_index(now_rest,list):
                for x in list:
                    if x[0]==now_rest and "관악구" in x[1]:
                        return list.index(x)
            time.sleep(2)
            click_name[rest_index(now_rest, data)].click()
            time.sleep(4)
            driver.switch_to.default_content()
            driver.switch_to.frame("entryIframe")
            review_list_2[review_list.index(li)][2] = driver.find_element_by_xpath('//*[@id="app-root"]/div/div[2]/div[1]/div/div/div[1]').text
            driver.quit()
        except:
            error_count+=1
            driver.quit()
print(review_list_2)
print(error_count)

[[0, '빠레뜨한남 샤로수길점', '빠레뜨한남 샤로수길점양식\n4.76/5방문자리뷰 393블로그리뷰 334'], [1, '샤로수길맛집 고기굽는사람들', '샤로수길맛집 고기굽는사람들육류,고기요리\n4.89/5방문자리뷰 1,159블로그리뷰 346'], [2, '베이컨시', '클릭 필요'], [3, '피자네버슬립스', '피자네버슬립스피자\n4.35/5방문자리뷰 134블로그리뷰 224'], [4, '라미스콘 앤 지민스통', '라미스콘 앤 지민스통카페,디저트\n4.32/5방문자리뷰 441블로그리뷰 174'], [5, '고요서울', '고요서울카페,디저트\n4.37/5방문자리뷰 127블로그리뷰 347'], [6, '문득', '문득이탈리아음식\n4.49/5방문자리뷰 175블로그리뷰 288'], [7, '9ounce Burger 낙성대점', '9ounce Burger 낙성대점햄버거\n4.43/5방문자리뷰 154블로그리뷰 668'], [8, '오 막회집 서울대점', '오 막회집 서울대점생선회\n4.45/5방문자리뷰 105블로그리뷰 310'], [9, '조선pub에디', '조선pub에디요리주점\n4.33/5방문자리뷰 70블로그리뷰 217'], [10, '소디스커피', '소디스커피카페\n4.42/5방문자리뷰 120블로그리뷰 336'], [11, 'kitchen& pub 몽중인', 'kitchen& pub 몽중인중식당\n4.44/5방문자리뷰 142블로그리뷰 112'], [12, '빽돈 본점', '빽돈 본점돼지고기구이\n4.54/5방문자리뷰 229블로그리뷰 370'], [13, '데일리오아시스 샤로수길점', '데일리오아시스 샤로수길점카페,디저트\n4.26/5방문자리뷰 284블로그리뷰 259'], [14, '섭지수산', '섭지수산이자카야\n4.31/5방문자리뷰 50블로그리뷰 40'], [15, '키요이 Bistro', '키요이 Bistro일식당\n4.59/5방문자리뷰 96블로그리뷰 275'], [16, '오신매운갈비찜 서울대입구점', '오신매운갈비찜 서울대입구점육류,고기요리\n4.48

In [9]:
review_list_3 = review_list_2
error_count=0
for li in review_list_2:
    if li[2] == "클릭 필요":
        try:
            now_rest = li[1]
            driver = webdriver.Chrome("./chromedriver\/chromedriver.exe")
            driver.get("https://map.naver.com/v5/search")
            time.sleep(3)
            search_box = driver.find_element_by_xpath('/html/body/app/layout/div[3]/div[2]/shrinkable-layout/div/app-base/search-input-box/div/div[1]/div/input')
            search_box.send_keys(now_rest)
            search_box.send_keys(Keys.ENTER)
            time.sleep(3)
            driver.find_element_by_xpath('//*[@id="container"]/shrinkable-layout/div/app-base/search-layout/div[1]/combined-search-list/salt-search-list/nm-external-frame-bridge').click()
            driver.switch_to.frame("searchIframe")
            click_name = driver.find_elements_by_xpath('/html/body/div[3]/div/div/div[1]/ul/li/div[2]/div[1]/a/span[1]')
            text_name = []
            for i in range(len(click_name)):
                text_name.append(click_name[i].text)
            adress = driver.find_elements_by_xpath('/html/body/div[3]/div/div/div[1]/ul/li/div[2]/div[2]/span/a/span[1]')
            for i in range(len(adress)):
                adress[i] = adress[i].text
            data = [] 
            for x, y in zip(text_name, adress):
                data.append([x, y])
            def rest_index(now_rest,list):
                for x in list:
                    if x[0]==now_rest and "관악구" in x[1]:
                        return list.index(x)
            time.sleep(2)
            click_name[rest_index(now_rest, data)].click()
            time.sleep(4)
            driver.switch_to.default_content()
            driver.switch_to.frame("entryIframe")
            review_list_3[review_list_2.index(li)][2] = driver.find_element_by_xpath('//*[@id="app-root"]/div/div[2]/div[1]/div/div/div[1]').text
            driver.quit()
        except:
            error_count+=1
            driver.quit()
print(review_list_3)
print(error_count)

[[0, '빠레뜨한남 샤로수길점', '빠레뜨한남 샤로수길점양식\n4.76/5방문자리뷰 393블로그리뷰 334'], [1, '샤로수길맛집 고기굽는사람들', '샤로수길맛집 고기굽는사람들육류,고기요리\n4.89/5방문자리뷰 1,159블로그리뷰 346'], [2, '베이컨시', '베이컨시이탈리아음식\n4.66/5방문자리뷰 124블로그리뷰 83'], [3, '피자네버슬립스', '피자네버슬립스피자\n4.35/5방문자리뷰 134블로그리뷰 224'], [4, '라미스콘 앤 지민스통', '라미스콘 앤 지민스통카페,디저트\n4.32/5방문자리뷰 441블로그리뷰 174'], [5, '고요서울', '고요서울카페,디저트\n4.37/5방문자리뷰 127블로그리뷰 347'], [6, '문득', '문득이탈리아음식\n4.49/5방문자리뷰 175블로그리뷰 288'], [7, '9ounce Burger 낙성대점', '9ounce Burger 낙성대점햄버거\n4.43/5방문자리뷰 154블로그리뷰 668'], [8, '오 막회집 서울대점', '오 막회집 서울대점생선회\n4.45/5방문자리뷰 105블로그리뷰 310'], [9, '조선pub에디', '조선pub에디요리주점\n4.33/5방문자리뷰 70블로그리뷰 217'], [10, '소디스커피', '소디스커피카페\n4.42/5방문자리뷰 120블로그리뷰 336'], [11, 'kitchen& pub 몽중인', 'kitchen& pub 몽중인중식당\n4.44/5방문자리뷰 142블로그리뷰 112'], [12, '빽돈 본점', '빽돈 본점돼지고기구이\n4.54/5방문자리뷰 229블로그리뷰 370'], [13, '데일리오아시스 샤로수길점', '데일리오아시스 샤로수길점카페,디저트\n4.26/5방문자리뷰 284블로그리뷰 259'], [14, '섭지수산', '섭지수산이자카야\n4.31/5방문자리뷰 50블로그리뷰 40'], [15, '키요이 Bistro', '키요이 Bistro일식당\n4.59/5방문자리뷰 96블로그리뷰 275'], [16, '오신매운갈비찜 서울대입구점'

In [10]:
import re
list_for_pd = []
for x in review_list_3:
    if x[2] != "클릭 필요":
        data = x[2][len(x[1]):]
        data2=data
        business=data2.split('\n')[0]
        p_visit = re.compile(r'(?<=방문자리뷰\s).+(?=블로그리뷰)')
        p_blog = re.compile(r'(?<=블로그리뷰\s).+')
        p_rating = re.compile(r'(?<=\n).+(?=/5)')
        m1=p_rating.search(data)
        m2=p_visit.search(data)
        m3=p_blog.search(data)
        if m1 == None:
            rating = 'None'
        else:
            rating = float(m1.group(0))
        if m2 == None:
            visit = 'None'
        else:
            visit = int(m2.group(0).replace(',',''))
        if m3 == None:
            blog = 'None'
        else:
            blog = int(m3.group(0).replace(',',''))
        list_for_pd.append([x[1],business,rating, visit, blog])
print(list_for_pd)

[['빠레뜨한남 샤로수길점', '양식', 4.76, 393, 334], ['샤로수길맛집 고기굽는사람들', '육류,고기요리', 4.89, 1159, 346], ['베이컨시', '이탈리아음식', 4.66, 124, 83], ['피자네버슬립스', '피자', 4.35, 134, 224], ['라미스콘 앤 지민스통', '카페,디저트', 4.32, 441, 174], ['고요서울', '카페,디저트', 4.37, 127, 347], ['문득', '이탈리아음식', 4.49, 175, 288], ['9ounce Burger 낙성대점', '햄버거', 4.43, 154, 668], ['오 막회집 서울대점', '생선회', 4.45, 105, 310], ['조선pub에디', '요리주점', 4.33, 70, 217], ['소디스커피', '카페', 4.42, 120, 336], ['kitchen& pub 몽중인', '중식당', 4.44, 142, 112], ['빽돈 본점', '돼지고기구이', 4.54, 229, 370], ['데일리오아시스 샤로수길점', '카페,디저트', 4.26, 284, 259], ['섭지수산', '이자카야', 4.31, 50, 40], ['키요이 Bistro', '일식당', 4.59, 96, 275], ['오신매운갈비찜 서울대입구점', '육류,고기요리', 4.48, 322, 351], ['낭만모로코', '양식', 4.6, 108, 181], ['남도반주', '한식', 4.83, 44, 10], ['충청삼겹', '육류,고기요리', 4.45, 142, 213], ['하울링데이즈', '햄버거', 4.56, 104, 172], ['테일러슬라이스피자', '피자', 4.38, 43, 199], ['안녕 와인', '와인', 4.13, 61, 169], ['아로이팟타이', '태국음식', 4.45, 94, 159], ['올더플레이트', '퓨전음식', 4.5, 155, 152], ['다르빛두번째이야기', '요리주점', 4.36, 28, 28], ['달첨시루', '떡카페', 4.5, 

In [11]:
df = pd.DataFrame(list_for_pd, columns=['name','category','rating','visitor review','blog review'])
df
df.to_csv('naver_ver5_x_restaurants.csv')